## Imports

In [28]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import folium
import tabula

Source link:

https://thetowerinfo.com/nyc-skyscrapers-under-construction-proposed/

## Read in and Set up data

In [29]:
df = pd.read_csv('NYC_skyscrapers - Sheet1.csv')
print(len(df))

23


## Geocode

In [30]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [31]:
df['geo_address'] = df['Address'] + 'New York, NY'

In [32]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['geo_address'].apply(geocode)

In [33]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

In [35]:
df.columns

Index(['Address', 'Height_meters', 'Timeframe', 'Architect', 'Developer',
       'Status', 'Color', 'geo_address', 'geocoded', 'lat', 'lon'],
      dtype='object')

In [36]:
def popup_html(row):
    Address = row['Address']
    Height = row['Height_meters']
    Timeframe = row['Timeframe']
    Architect = row['Architect']
    Developer = row['Developer']
    Status = row['Status']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>Height (meters): </strong>{}'''.format(Height) + '''<br>
    <strong>Timeframe: </strong>{}'''.format(Timeframe) + '''<br>
    <strong>Architect: </strong>{}'''.format(Architect) + '''<br>
    <strong>Developer: </strong>{}'''.format(Developer) + '''<br>
    <strong>Status: </strong>{}'''.format(Status) + '''<br>
    </html>
    '''
    return html

In [38]:
df.columns

Index(['Address', 'Height_meters', 'Timeframe', 'Architect', 'Developer',
       'Status', 'Color', 'geo_address', 'geocoded', 'lat', 'lon'],
      dtype='object')

In [43]:
### Create map container ###
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=11.5,tiles=None)

### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"NYC Proposed and Under Construction Skyscrapers")
m.get_root().html.add_child(folium.Element(title_html))

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='Under Construction')
fg_orange = folium.FeatureGroup(name='Proposed')

for index, row in df.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['Color'] == 'blue':
        marker = folium.Marker(
            location=[row['lat'], row['lon']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['Color']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['lat'], row['lon']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['Color']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)

# Display map
m

In [44]:
m.save('index.html')

## Map URL Snagger

In [45]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [46]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/NYCskyScrapers
